Program to convert a FreePlane mindmap to a regular PDF document.

We are using pylatex module of Python to generate LaTeX document and then from it, generate PDF.
We are using freeplane-io module of Python to parse and manage the mindmap (XML) file.

In [238]:
"""
A program to convert Freeplane based mindmaps to corresponding PDF documents, but with style! :)
If the mindmap is prepared, following certain conventions, you can produce print-quality PDF
documents using fp-convert.

Author: K Raghu Prasad <raghuprasad AT duck.com>
Copyright: ©2024-25 K Raghu Prasad <raghuprasad AT duck.com>
Licence: Apache 2.0 Licence
"""
import os, freeplane
from pylatex import (
    Alignat,
    Axis,
    Command,
    Description,
    Document,
    Figure,
    FlushLeft,
    Foot,
    Head,
    Itemize,
    Label,
    LargeText,
    LineBreak,
    Math,
    Matrix,
    MediumText,
    MiniPage,
    NewLine,
    NoEscape,
    Package,
    PageStyle,
    Plot,
    Section,
    Subsection,
    Subsubsection,
    Tabular,
    TikZ,
    simple_page_number,
)
from pylatex.section import Paragraph
from pylatex.position import VerticalSpace
from pylatex.utils import bold, italic, verbatim
from pytablewriter import LatexTableWriter


class TableTheme:
    header_color = "Apricot",
    rowcolor_1 = "gray!25"
    rowcolor_2 = "white"
    line_color = "red"

 
class Theme:
    table = TableTheme()


class HeaderConfig:
    headline = True
    headline_color = "red"
    header_left = ""
    header_center = ""
    header_right = ""
    
 
class FooterConfig:
    footline = True
    footline_color = "red"
    footer_left = ""
    footer_center = ""
    footer_right = ""


class Config:
    header = HeaderConfig()
    footer = FooterConfig()
    
    
theme = Theme()
config = Config()

geometry={"top": "3in", "left": "1.5in", "right": "1.5in", "bottom": "2in"}
#geometry={"hmargin": "2.5cm","vmargin": "3cm","bindingoffset": "0.5cm"}

class FPDocument(Document):
    """
    Document class to build a Freeplane Document (from a Freeplane Mindmap).
    """
    def __init__(self, geometry=geometry, theme=None, config=None):
        #, title="<Missing Title>", author="<Missing Author>", date=NoEscape(r"\today")):
        super().__init__()
        
        # Maintainer a container with ids of the nodes which are already processed
        self.processed_nodes = set()
        
        # Add required packages
        self.packages.append(Package('geometry'))
        self.packages.append(Package('xcolor', options=("dvipsnames", "table")))
        self.packages.append(Package('tcolorbox', options="most"))
        self.packages.append(Package('placeins', options=("section",)))
        self.packages.append(Package('hyperref'))
        self.packages.append(Package('fontenc', options="T1"))
        #self.packages.append(Package('fontspec'))
        self.preamble.append(NoEscape(r"""
\hypersetup{colorlinks,linkcolor={red!50!black},
citecolor={blue!50!black},
urlcolor={blue!80!black}}"""))
        self.preamble.append(NoEscape(f"""
\\geometry{{
a4paper,
%total={{170mm,257mm}},
left={geometry["left"]},
right={geometry["right"]},
top={geometry["top"]},
bottom={geometry["bottom"]},
}}"""))
        
        # Apply theme
        if theme:
            self.apply_theme(theme)
        
        
    def generate_header(self, title="<Missing Title>", author="<Missing Auther>",
                        company="<Missing Organization", date=NoEscape(r"\today"),
                        lfooter="Left Footer", rfooter="Right Footer"):
        self.preamble.append(Command("title", title))
        self.preamble.append(Command("author", author))
        self.preamble.append(Command("date", date))
        
        # Add document header
        header = PageStyle("header",
                           header_thickness=0.4,
                           footer_thickness=0.4,
                           data=NoEscape(r'''
\renewcommand{\headrule}{\color{red}\hrule width \headwidth height \headrulewidth}  % Red line
\renewcommand{\footrule}{\color{red}\hrule width \headwidth height \footrulewidth}  % Red line
'''))
        
        # Create left header
        with header.create(Head("L")):
            #header.append("Page date: ")
            #header.append(LineBreak())
            header.append("Logo-1")
            
        # Create center header
        with header.create(Head("C", data=NoEscape("\\normalcolor"))):
            header.append(company)
            
        # Create right header
        with header.create(Head("R", data=NoEscape("\\normalcolor"))):
        #    header.append(NoEscape(r'\small{©2024-25 Example Corporation}', data=NoEscape("\\normalcolor")))
            header.append(NoEscape("\\small{Confidential}"))
            
        # Create left footer
        with header.create(Foot("L", data=NoEscape("\\normalcolor"))):
            header.append(NoEscape(r'\tiny{Generated by \href{https://www.python.org}{fp-convert}.}'))
            
        # Create center footer
        with header.create(Foot("C", data=NoEscape("\\normalcolor"))):
            header.append("Logo-2")
            
        # Create right footer
        with header.create(Foot("R", data=NoEscape("\\normalcolor"))):
            #header.append(simple_page_number())
            header.append(NoEscape("\\small{Page \\thepage\- of \pageref*{LastPage}}"))

        self.preamble.append(header)
        self.change_document_style("header")

        self.append(NoEscape(r"\maketitle"))
        
        # Add Heading
        #with self.create(MiniPage(align="c")):
        #    self.append(LargeText(bold("Title")))
        #    self.append(LineBreak())
        #    self.append(MediumText(bold("As at:")))
        

    def apply_theme(self, theme):
        self.theme = theme
        self.append(NoEscape(r'\rowcolors{2}{'+theme.table.rowcolor_1+r'}{'+theme.table.rowcolor_2+r'}'))
    

Prepare some utility functions to build certain sections of the document using Freeplane Nodes.

In [239]:

def get_label(prefix, id):
    """
    Replace _ with : in the ID of the nodes created by FP.
    """
    return prefix+id.replace("_", ":")


def build_figure(node, doc):
    """
    Build and return a LaTeX figure element using the supplied node.
    """
    ret = list()

    if node.imagepath:
        fig = Figure(position='!htb')
        fig.add_image(node.imagepath, width=NoEscape(r'0.7\textwidth'), placement=NoEscape(r'\centering'))
        fig.add_caption(node)
        fig.append(NoEscape(r'\label{' + get_label("FIG:", node.id + '}')))
        ret.append(fig)
        
    # Now check if the same node has other elements to be created.
    if "list" in node.icons:  # There are lists to be rendered now under this section
        lst = build_recursive_list(node, doc, 1)
        ret.append(lst)
    if ret:
        return ret
    return ""
        

def build_verbatim_list(node, doc):
    """
    Build a list of parts with the contents of the children printed in verbatim mode.
    """
    #print(f"[{node.id}]", node)
    #print(f"[children]", node.children)
    doc.processed_nodes.add(node.id)
    if node.children:
        itmz =  Itemize()
        for child in node.children:  # Every item-element starts with [] to avoid bullets
            p = NoEscape('[]\\begin{verbatim}') + NoEscape(child) + NoEscape('\\end{verbatim}')
            if child.notes:
                p = f"""{p}%
                {child.notes}"""
            itmz.add_item(NoEscape(p))
        return itmz
    return ""
    
    
def append_notes_if_exists(node, item, prefix=None, suffix=None):
    """
    Append a newline prefixed note-segment to the supplied item-segment, if notes exist.
    """
    if node.notes:
        if prefix:
            item.append(prefix)
        item.append(node.notes)
        if suffix:
            item.append(suffix)

 
def build_recursive_list(node, doc, level):
    """
    Build and return a recursive list of lists as long as child nodes are present.
    """
    doc.processed_nodes.add(node.id)
    if node.children:
        itmz =  Itemize()
        for child in node.children:
            # For the purpose of debugging only
            #print("  "*level, f"[{child.id}]", child)
            #print("  "*level, f"[imagepath]", child.imagepath)
            #print("  "*level, f"[icons]", child.icons)
            doc.processed_nodes.add(child.id)
            content = str(child).split(":", 1)
            if len(content) == 2:
                itmz.add_item(NoEscape(f"{bold(content[0])}: {content[1]}"))
            elif child.children:
                itmz.add_item(NoEscape(f"{bold(child)}"))
            else:
                itmz.add_item(child)
            append_notes_if_exists(child, itmz, prefix=NewLine())
            if child.children:
                # Check if children should be formatted verbatim
                if 'links/file/json' in child.icons or \
                   'links/file/xml' in child.icons or \
                   'links/file/html' in child.icons:
                    itmz.append(build_verbatim_list(child, doc))
                else:  # Expecting a plain list, or list of list, or list of lists ...
                    itmz.append(build_recursive_list(child, doc, level+1))
        return itmz
    return ""


def build_paragraph_per_line(content, doc):
    """
    If the supplied content is newline separated, then each line is treated as a standalone
    paragraph.
    """
    lines = [str.strip(l) for l in str.strip(content).split("\n") if str.strip(l)]
    #print(f"lines: {lines}")
    if len(lines) == 1:
        return lines[0]
    ret = list()
    for line in lines:
        ret.append(NoEscape(f"\\flushleft {line}"))
    return "".join(ret)

    
def build_table_and_notelist(node, doc):
    """
    Build a tabular layout using the tree of information obtained from the supplied of node.
    """
    doc.processed_nodes.add(node.id)
    #print(doc.processed_nodes)
    
    if node.children:
        col1 = dict()  # Collection of table-data
        notes = list() # Collection of notes (if they exist)

        for field in node.children:
            if field:
                #print(f"Field is {field}")
                doc.processed_nodes.add(field.id)
                col1[str(field)] = {str.strip(str(d).split(":")[0]): str.strip(str(d).split(":")[1]) for d in field.children}
                if field.notes:
                    notes.append((field, field.notes))
                
        col_hdrs = sorted(list({e for d in col1.values() for e in d.keys()}))

        tab = Tabular("l" * (1+len(col_hdrs)))
        #tab.add_caption(node, label=get_label("TAB:", node.id))
        tab.add_hline(color=doc.theme.table.line_color)
        row = list(" ")
        row.extend([bold(hdr) for hdr in col_hdrs])
        tab.add_row(*row, color=doc.theme.table.header_color, strict=True)
        tab.add_hline(color=doc.theme.table.line_color)
        for field in sorted(col1.keys()):
            row = [field,]
            for col in col_hdrs:
                row.append(col1[field].get(col, ""))
            tab.add_row(row)
        tab.add_hline(color=doc.theme.table.line_color)
        if notes:
            itmz = Itemize()
            for field, note in notes:
                itmz.add_item(NoEscape(f"{bold(field)}: {build_paragraph_per_line(str(note), doc)}"))
            return (tab, itmz)
        return (tab, )
    return ""


def add_to_blocks(block, item_list):
    """
    Append items from item_list into the block.
    """
    for item in item_list:
        block.append(item)
        

def traverse_children(node, indent, doc):
    """
    Traverse the node-tree and build sections, subsections, and further sections and
    tables based on the depth of the node under process.
    """
    if node:
        #For the purpose of debugging only
        #print("  "*indent, f"[{node.id}]", node)
        #if node.imagepath:
        #    print("  "*indent, f"[{node.id}]", node.imagepath)
        #if node.icons:
        #    print("  "*indent, f"[{node.id}]", node.icons)
         
        if node.id in doc.processed_nodes:
            # This node is processed already via some look-ahead helper functions
            return
        
        if indent == 1:
            blocks = [Section(f"{node}", label=Label(get_label("SEC:", node.id))),]
        elif indent == 2:
            blocks = [Subsection(f"{node}", label=Label(get_label("SEC:", node.id))),]
        elif indent == 3:
            blocks = [Subsubsection(f"{node}", label=Label(get_label("SEC:", node.id))),]
        elif indent == 4:
            blocks = [Paragraph(NoEscape(f"{node}\\newline\\noindent"), label=Label(get_label("SEC:", node.id))),]
        else:
            return

        if node.children:  # Non-section specific things are handled here
            if node.icons and 'links/file/generic' in node.icons:  # Table is to be built
                append_notes_if_exists(node, blocks, suffix=NewLine())
                blocks.extend(build_table_and_notelist(node, doc))
            elif node.icons and 'list' in node.icons:  # List is to be built
                append_notes_if_exists(node, blocks)
                blocks.append(build_recursive_list(node, doc, 1))
            elif node.icons and 'image' in node.icons:  # List is to be built
                append_notes_if_exists(node, blocks)
                for child in node.children:
                    add_to_blocks(blocks, build_figure(child, doc))
        else:
            append_notes_if_exists(node, blocks)
        
        with doc.create(blocks[0]):
            #if node.notes:
            #    doc.append(node.notes)

            #if len(blocks) > 1:
            #    with doc.create(FlushLeft()) as laligned:
            #        for element in blocks[1:]:
            #            laligned.append(element)
            if len(blocks) > 1:
                for element in blocks[1:]:
                    doc.append(element)
        doc.processed_nodes.add(node.id)  # Register this node as already processed
        for child in node.children:
            traverse_children(child, indent+1, doc)

Then we access the root node and start iterating the children there onwards.

In [243]:

if __name__ == '__main__':
    mindmap_file_path = "./FPConvertTest.mm"
    mm = freeplane.Mindmap(mindmap_file_path)
    node = mm.rootnode

    theme = Theme()
    config = Config()
    #doc = FPDocument(theme=theme)
    geometry = {"left": "1.5in", "right": "1.5in", "top": "1.5in", "bottom": "1.5in"}
    doc = FPDocument(geometry=geometry, theme=theme)
    #doc.packages.append(Package('berasans', options="scaled"))
    #doc.packages.append(Package('AlegreyaSans', options="sfdefault"))
    #doc.packages.append(Package('comicneue', options=("default", "angular")))
    #doc.packages.append(Package("montserrat", options=("defaultfam","tabular","lining")))
    #doc.packages.append(Package('nunito', options=("tabular","lining")))
    #doc.packages.append(Package('nunito'))
    #doc.packages.append(Package('nimbussans'))  # *** Looks OK ***
    #doc.packages.append(Package('nunito', options="lining"))  # *** Looks OK ***
    #doc.packages.append(Package('noto', options="sfdefault"))  # *** Looks OK ***
    doc.packages.append(Package('roboto', options="sfdefault")) # *** Looks Great ***
    
    # Apply custom text in headers and footers
    doc.generate_header(node, "Ravi Prakash", "Blooper Production Corporation")
    
    #with doc.create(Section("Introduction")):
    #    doc.append(f"Document retrieved from mindmap {mindmap_file_path}.\nRoot node is {node}.")
    for child in node.children:
        traverse_children(child, 1, doc)
    doc.generate_pdf(node.id, clean_tex=False)